In [1]:
#Dependencies

import xlrd
import pprint

# Pandas
import pandas as pd
import numpy as np


# SQL Alchemy
from sqlalchemy import create_engine
# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base
# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float


In [2]:
# # Example Create a "Metadata" Layer That Abstracts our SQL Database
# Base.metadata.create_all(engine)

# # Use this to clear out the db
# Base.metadata.drop_all(engine)

# # Create a Session Object to Connect to DB
# from sqlalchemy.orm import Session
# session = Session(bind=engine)


In [3]:
#Read in data from unstructured Excel file
book = xlrd.open_workbook('Resources/SNAP/FY19.xls')
sheet = book.sheet_by_name('NERO')

print (dir(sheet))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_cell_attr_to_xfx', '_cell_types', '_cell_values', '_cell_xf_indexes', '_dimncols', '_dimnrows', '_first_full_rowx', '_ixfe', '_maxdatacolx', '_maxdatarowx', '_position', '_repr_these', '_xf_index_stats', '_xf_index_to_xl_type_map', 'automatic_grid_line_colour', 'bf', 'biff_version', 'book', 'bt', 'cached_normal_view_mag_factor', 'cached_page_break_preview_mag_factor', 'cell', 'cell_note_map', 'cell_type', 'cell_value', 'cell_xf_index', 'col', 'col_label_ranges', 'col_slice', 'col_types', 'col_values', 'colinfo_map', 'columns_from_right_to_left', 'computed_column_width', 'cooked_normal_view_mag_factor', 'cooked_page_break_preview_mag_factor', 'de

In [4]:
#Example Counts # of rows in your data
print (sheet.nrows)

145


In [5]:
# Example Shows the names of all of the sheets in the Excel Workbook

for sheet in book.sheets():
    print (sheet.name)

NERO
MARO
SERO
MWRO
SWRO
MPRO
WRO
US Summary


In [6]:
#This is what we're trying to pull from the data and how we want it to look
data_test = {
        ('Connecticut', 'October 2018'): {
            'household_participation': 1,
            'individual_participation': 1,
            'total_cost' : 1,
            'cost_by_household' : 1,
            'cost_by_individual': 1,
        },
        ('Connecticut', 'November 2018'): {
            'household_participation': 1,
            'individual_participation': 1,
            'total_cost' : 1,
            'cost_by_household' : 1,
            'cost_by_individual': 1,
            }
        }

In [7]:
pd.DataFrame.from_dict(data_test, orient = 'index')

household_participation  individual_participation  \
Connecticut October 2018                         1                         1   
            November 2018                        1                         1   

                           total_cost  cost_by_household  cost_by_individual  
Connecticut October 2018            1                  1                   1  
            November 2018           1                  1                   1

In [8]:
for i in range(7, sheet.nrows):
    row = sheet.row_values(i)
    
    print (i,row)

7 ['US Summary', '', '', '', '', '']
8 ['Oct 2018', 19722719.0, 39575279.0, 4904519214.0, 248.6736, 123.9289]
9 ['Nov 2018', 19537752.0, 39131093.0, 4798871042.0, 245.6204, 122.6358]
10 ['Dec 2018', 19350569.0, 38655988.0, 4735685303.0, 244.7311, 122.5085]
11 ['Jan 2019 /2', 20438289.0, 40627707.0, 8808799831.0, 430.995, 216.8175]
12 ['Feb 2019 /2', 3695943.0, 7422273.0, 619365505.0, 167.5798, 83.4469]
13 ['Mar 2019', 18942667.0, 37664927.0, 4466551286.0, 235.7932, 118.5865]
14 ['Apr 2019', 18844455.0, 37422372.0, 4520914707.0, 239.9069, 120.8078]
15 ['May 2019', 18833780.0, 37381135.0, 4543639171.0, 241.2495, 121.549]
16 ['Jun 2019', 18973316.0, 37532817.0, 4552219037.0, 239.9274, 121.2864]
17 ['Jul 2019', 19026938.0, 37602856.0, 4560929145.0, 239.709, 121.2921]
18 ['Aug 2019', 19120797.0, 37777171.0, 4574276320.0, 239.2304, 121.0857]
19 ['Sep 2019', 19081687.0, 37637717.0, 4536111652.0, 237.7207, 120.5204]
20 ['Total', 17964076.0, 35702611.25, 55621882213.0, 258.023675, 129.826833333

In [9]:
data = {}
for i in range(7, sheet.nrows):#Start at 7 b/c data starts here
    #RULES of this For Loop:
    row = sheet.row_values(i)
    
    if i in [x for x in range(7, sheet.nrows, 15)]: # x for x in range creates list comprehnsion to pull out all states incremented to 15th row
        #for the rows that have the state names
        state = row[0] # take what's in 1st index and call it variable "state"
        if state == 'Footnotes:':
            break
        else:
            continue # if it's not the end (break) continue to next row
    if i in [x for x in range(21, sheet.nrows, 15)]: # is this one of these iterations,if it is then skip it, if not, continue to next row? These are the blank lines
        continue #go to the next row
    #Read each ROW BY ROW
    month = row[0]
    household_participation = row[1]
    individual_participation = row[2]
    total_cost = row[3]
    cost_by_household = row[4]
    cost_by_individual = row[5]
    
    #create tupples to see how data comes out
    #data[(stare, month)] = (household_participation, individual_participation, total_cost, cost_by_household, cost_by_individual)
    
    #create a dictionary out of the data instead of tupples so we can use the data
    data[(state, month)] = {'household_participation': household_participation,
                            'individual_participation': individual_participation,
                            'total_cost': total_cost,
                            'cost_by_household': cost_by_household,
                            'cost_by_individual': cost_by_individual
        } 

    
pprint.pprint(data)

{('US Summary', 'Apr 2019'): {'cost_by_household': 239.9069,
                              'cost_by_individual': 120.8078,
                              'household_participation': 18844455.0,
                              'individual_participation': 37422372.0,
                              'total_cost': 4520914707.0},
 ('US Summary', 'Aug 2019'): {'cost_by_household': 239.2304,
                              'cost_by_individual': 121.0857,
                              'household_participation': 19120797.0,
                              'individual_participation': 37777171.0,
                              'total_cost': 4574276320.0},
 ('US Summary', 'Dec 2018'): {'cost_by_household': 244.7311,
                              'cost_by_individual': 122.5085,
                              'household_participation': 19350569.0,
                              'individual_participation': 38655988.0,
                              'total_cost': 4735685303.0},
 ('US Summary', 'Feb 2019 /2'): {'cos